In [1]:
import pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
# !{sys.executable} -m pip install --extra-index-url https://test.pypi.org/project/ protabank --upgrade

import protabank as pb

In [3]:
connection = pb.Connection('', '')
study = connection.get_study('MPSZ3Nkk3')
#study.sequences
#study.labels

array([['Brightness', 'SD of brightness'],
       ['1.30102987867', 'nan'],
       ['3.32511555279', 'nan'],
       ...,
       ['1.3007288107', 'nan'],
       ['3.67811388055', 'nan'],
       ['1.50991593608', 'nan']], dtype='<U32')

In [5]:
labels = pd.DataFrame(study.labels)
new_header = labels.iloc[0] #grab the first row for the header
labels = labels[1:] #take the data less the header row
labels.columns = new_header #set the header row as the df header
labels.head()

,Brightness,SD of brightness
1,1.30102987867,nan
2,3.32511555279,nan
3,3.17207795627,nan
4,3.64149927703,0.0884949444496
5,1.5799740943,nan


In [24]:
# this converts the brightness into a list of strings
brights= np.array(labels['Brightness'])
brights = [float(entry) for entry in brights]

In [27]:
seqs = pd.DataFrame(study.sequences)
seqs.head()

,0,1,2,3,4,5,6,7,8,9,...,228,229,230,231,232,233,234,235,236,237
0,11,16,9,6,4,4,10,5,17,6,...,8,17,7,6,11,3,4,10,20,9
1,11,16,9,6,4,4,10,5,17,6,...,8,17,7,6,11,3,4,10,20,9
2,11,16,9,6,4,4,10,5,17,6,...,8,17,7,6,11,3,4,10,20,9
3,11,16,9,6,4,4,10,5,17,6,...,8,17,7,6,11,3,4,10,20,9
4,11,16,9,6,4,4,10,5,17,6,...,8,17,7,6,11,3,4,10,20,9


In [81]:
# this converts our categorial sequences into a list of one hot lists
gfp_len = 238
onehotseqs = []
for seq in study.sequences:
    seq = np.array(seq-1)
    onehotseq = np.zeros((gfp_len,20))
    onehotseq[np.arange(gfp_len),seq]=1
    onehotseqs.append(onehotseq)

In [167]:
# this gets the one hot enconded sequences and for each sequence concatenas teh amino acids (since each aa is a onehot list of len 20)
# the final product has dimensions of 51714 proteins by 238aa*20types =  (51714, 4760)

X = []
for ohseq in onehotseqs:
    concat = []
    for aa in ohseq:
        concat.extend(aa)
    X.append(concat)
    #print(concat)
    #break
X = np.array(X)
np.shape(X)

(51714, 4760)

In [168]:
Y = np.array(list(brights))
len(Y)

51714

In [108]:
# but for now let's use odd indexes for training and even for testing:
train_indices = np.array(range(1,nproteins,2)) 
test_indices = np.array(range(0,nproteins,2))
print("TRAIN INDEX:", train_indices,'\n', "TEST INDEX:", test_indices)

TRAIN INDEX: [    1     3     5 ... 51709 51711 51713] 
 TEST INDEX: [    0     2     4 ... 51708 51710 51712]


## Define the models
Here we're trying a simple logistic regression with mean squared error for the loss

In [165]:
# Build the models for each taxonomy rank level
BATCH_SIZE = 20000 # batchsize = half of the data
EPOCHS = 4
STEPS_PER_EPOCH = 5 # len(train_indices) // BATCH_SIZE

# number of cell types (labels) in taxonomy rank
output_dim = 1
#number of genes
input_dim = np.shape(X)[1]
model = Sequential() 
model.add(Dense(output_dim, input_dim=input_dim, activation='softmax')) 

#Compile the model
model.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['accuracy']) 

## Train the model
We just train for a few epochs to see if the code runs

In [174]:
print ('HALF-HALF DATA SPLIT FOR TRAINING AND VALIDATION OF LOGREG CLASSIFIER')
history = model.fit(
        x = np.array(X)[train_indices],
        y = np.array(Y)[train_indices],
        validation_data=(X[test_indices], Y[test_indices]),
        validation_steps=len(test_indices) // BATCH_SIZE,
        epochs=5,
        steps_per_epoch= len(train_indices) // BATCH_SIZE)



HALF-HALF DATA SPLIT FOR TRAINING AND VALIDATION OF LOGREG CLASSIFIER
Train on 25857 samples, validate on 25857 samples
Epoch 1/5
2/2 [==============================] - 1s 413ms/step - loss: 3.8491 - acc: 0.0000e+00 - val_loss: 3.8896 - val_acc: 0.0000e+00
Epoch 2/5
2/2 [==============================] - 1s 429ms/step - loss: 3.8491 - acc: 0.0000e+00 - val_loss: 3.8896 - val_acc: 0.0000e+00
Epoch 3/5
2/2 [==============================] - 1s 388ms/step - loss: 3.8491 - acc: 0.0000e+00 - val_loss: 3.8896 - val_acc: 0.0000e+00
Epoch 4/5
2/2 [==============================] - 1s 456ms/step - loss: 3.8491 - acc: 0.0000e+00 - val_loss: 3.8896 - val_acc: 0.0000e+00
Epoch 5/5
2/2 [==============================] - 1s 433ms/step - loss: 3.8491 - acc: 0.0000e+00 - val_loss: 3.8896 - val_acc: 0.0000e+00
